# 1. Data Acquisition

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Create a script to connect to PostgreSQL, create a table, insert values and then finally read from it 
#### (Normally we could have done this just by reading from the .csv file but as per the project, it is requested to use PostgreSQL as a datasource)

In [2]:
#df = pd.read_csv("heart_extended.csv")

In [3]:
import psycopg2
import psycopg2.extras

In [4]:
hostname = 'localhost'
database = 'heart'
username = 'postgres'
pwd = 'postgres'
port_id = 5432

In [5]:
# Create the table

conn = None

try:
    with psycopg2.connect(
                host = hostname,
                dbname = database,
                user = username,
                password = pwd,
                port = port_id) as conn:

        with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as cur:

            cur.execute('DROP TABLE IF EXISTS heart')

            create_script = ''' CREATE TABLE IF NOT EXISTS heart (
                                    id              serial PRIMARY KEY,
                                    Age             int,
                                    Sex             varchar(1),
                                    ChestPainType   varchar(5),
                                    RestingBP       int,
                                    Cholesterol     int,
                                    FastingBS       int,
                                    RestingECG      varchar(10),
                                    MaxHR           int,
                                    ExerciseAngina  varchar(1),
                                    Oldpeak         real,
                                    ST_Slope        varchar(5),
                                    HeartDisease    int) '''
            
            cur.execute(create_script)
            
except Exception as error:
    print(error)
    
finally:
    if conn is not None:
        conn.close()

In [6]:
# Insert data into the table by using heart.sql file
try:
    with psycopg2.connect(
                host = hostname,
                dbname = database,
                user = username,
                password = pwd,
                port = port_id) as conn:

        with conn.cursor(cursor_factory=psycopg2.extras.DictCursor) as cur:
            
            f = open("heart.sql",'r')
            insert_script  = f.read()
            cur.execute(insert_script)
            
except Exception as error:
    print(error)
    
finally:
    if conn is not None:
        conn.close()

In [7]:
# Read data from the table

try:
    conn = psycopg2.connect(host = hostname,
                dbname = database,
                user = username,
                password = pwd,
                port = port_id)
    
    cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    cur.execute('SELECT * FROM heart')
    
    data = cur.fetchall()
    print(data[0])
    
    #for record in cur.fetchall():
    #            print(record)
    
except Exception as error:
    print(error)

[1, 40, 'M', 'ATA', 140, 289, 0, 'Normal', 172, 'N', 0.0, 'Up', 0]


In [8]:
df = pd.DataFrame(data, columns=['id','Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease'])

In [9]:
df.drop('id', axis=1)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [10]:
df.nunique()

id                918
Age                50
Sex                 2
ChestPainType       4
RestingBP          67
Cholesterol       222
FastingBS           2
RestingECG          3
MaxHR             119
ExerciseAngina      2
Oldpeak            53
ST_Slope            3
HeartDisease        2
dtype: int64

# The Attributess include:
- Age: age of the patient [years]
- Sex: sex of the patient [M: Male, F: Female]
- ChestPainType: chest pain type [TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic]
- RestingBP: resting blood pressure [mm Hg]
- Cholesterol: serum cholesterol [mm/dl]
- FastingBS: fasting blood sugar [1: if FastingBS > 120 mg/dl, 0: otherwise]
- RestingECG: resting electrocardiogram results [Normal: Normal, ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), LVH: showing probable or definite left ventricular hypertrophy by Estes' criteria]
- MaxHR: maximum heart rate achieved [Numeric value between 60 and 202]
- ExerciseAngina: exercise-induced angina [Y: Yes, N: No]
- Oldpeak: oldpeak = ST [Numeric value measured in depression]
- ST_Slope: the slope of the peak exercise ST segment [Up: upsloping, Flat: flat, Down: downsloping]
- HeartDisease: output class [1: heart disease, 0: Normal]

# 2. Preprocessing

## 2.1 Check for null values

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              918 non-null    int64  
 1   Age             918 non-null    int64  
 2   Sex             918 non-null    object 
 3   ChestPainType   918 non-null    object 
 4   RestingBP       918 non-null    int64  
 5   Cholesterol     918 non-null    int64  
 6   FastingBS       918 non-null    int64  
 7   RestingECG      918 non-null    object 
 8   MaxHR           918 non-null    int64  
 9   ExerciseAngina  918 non-null    object 
 10  Oldpeak         918 non-null    float64
 11  ST_Slope        918 non-null    object 
 12  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 93.4+ KB


In [12]:
df.isnull().sum()

id                0
Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

In [13]:
# There are not any null values in the data set

## 2.2 Handling Categorical Features

In [14]:
cat_cols = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

In [15]:
for col in cat_cols:
    print(f"The distribution of categorical values in the {col} is : ")
    print(df[col].value_counts())

The distribution of categorical values in the Sex is : 
M    725
F    193
Name: Sex, dtype: int64
The distribution of categorical values in the ChestPainType is : 
ASY    496
NAP    203
ATA    173
TA      46
Name: ChestPainType, dtype: int64
The distribution of categorical values in the RestingECG is : 
Normal    552
LVH       188
ST        178
Name: RestingECG, dtype: int64
The distribution of categorical values in the ExerciseAngina is : 
N    547
Y    371
Name: ExerciseAngina, dtype: int64
The distribution of categorical values in the ST_Slope is : 
Flat    460
Up      395
Down     63
Name: ST_Slope, dtype: int64


### Kategorik değerlerin dönüşümünde temelde 2 tip yöntem var bunlar:

* Label Encoding
* One Hot Encoding (pd.get_dummies()!)

#### Biz bu çalışmada LabelEncoding tekniği ile verimizi ön işleme aşamasına tabi tuttuk. Ancak bu her algoritma için tavsiye edilmez. 

![scaling](scaling_encoding.png)

In [16]:
from sklearn.preprocessing import LabelEncoder

df_encoded = df.apply(LabelEncoder().fit_transform)
df_encoded.head()

,id,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,0,12,1,1,41,147,0,1,98,0,10,2,0
1,1,21,0,2,55,40,0,1,82,0,20,1,1
2,2,9,1,1,31,141,0,2,25,0,10,2,0
3,3,20,0,0,39,72,0,1,34,1,25,1,1
4,4,26,1,2,49,53,0,1,48,0,10,2,0


In [17]:
for col in cat_cols:
    print(f"The distribution of categorical values in the {col} is : ")
    print(df_encoded[col].value_counts())

The distribution of categorical values in the Sex is : 
1    725
0    193
Name: Sex, dtype: int64
The distribution of categorical values in the ChestPainType is : 
0    496
2    203
1    173
3     46
Name: ChestPainType, dtype: int64
The distribution of categorical values in the RestingECG is : 
1    552
0    188
2    178
Name: RestingECG, dtype: int64
The distribution of categorical values in the ExerciseAngina is : 
0    547
1    371
Name: ExerciseAngina, dtype: int64
The distribution of categorical values in the ST_Slope is : 
1    460
2    395
0     63
Name: ST_Slope, dtype: int64
